# Neural and Evolutionary Leaning Project 

Group members: 

- Iris Moreira - 20240659
- Leonardo Di Caterina - 20240485
- Rafael Borges - 20240497

## Fourth Delivery - NN

In [8]:
# Standard library imports
import os
import random
import statistics
import time
from collections import defaultdict
import itertools
from itertools import product

# Third-party imports
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm

# Slim-GSGP imports
from slim_gsgp.datasets.data_loader import load_pandas_df
from slim_gsgp.utils.utils import train_test_split
#from slim_gsgp.main_gp import gp
#from slim_gsgp.main_gsgp import gsgp
#from slim_gsgp.main_slim import slim
#from slim_gsgp.evaluators.fitness_functions import rmse


In [9]:
os.chdir(os.path.join(os.getcwd(), os.pardir))
from utils.grid_search import gp_nested_cross_validation, fit_model_GridSearch, group_and_median_rmse
from utils.visualization_funcs import *
%cd notebooks/

c:\Users\irism\OneDrive - NOVAIMS\Msc-NEL\Neural_Evo_Learn\notebooks


In [3]:
class RMSELoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.mse = nn.MSELoss()
        
    def forward(self,yhat,y):
        return torch.sqrt(self.mse(yhat,y))

criterion = RMSELoss()

In [4]:
class Net_Arc(nn.Module):
    
    def __init__(self, input_size=2, output_size=1,
                 hidden_layer_sizes:tuple = (3,4,5), activation=nn.ReLU()):
        
        super(Net_Arc, self).__init__()
        #
        # 1. 1. Network architecture
        
        self.add_module(f'fc{1}', nn.Linear(input_size, hidden_layer_sizes[0]))
        
        for i in range(1,len(hidden_layer_sizes)):
            self.add_module(f'fc{i+1}', nn.Linear(hidden_layer_sizes[i-1], hidden_layer_sizes[i]))
        
        self.add_module(f'fc{len(hidden_layer_sizes) +1 }', nn.Linear(hidden_layer_sizes[-1], output_size))

        # Weights initialisation
        # The apply method applies the function passed as the apply() argument
        # to each element in the object, that in this case is the neural network.
        self.apply(self._init_weights)
        # Store the parameters
        self.input_size = input_size
        self.output_size = output_size
        self.hidden_layer_sizes = hidden_layer_sizes
        self.activation = activation
        
        self.n_forward_calls = 0
        
    #
    # 1. 2. Weights and bias initialization
    #
    def _init_weights(self, attribute):
        if isinstance(attribute, nn.Linear):
          torch.nn.init.xavier_uniform_(attribute.weight)
          torch.nn.init.zeros_(attribute.bias)
    #
    # 1. 3. Forward pass
    """    
    def forward(self, x):
        # For each layer, the output will be the ReLu activation applied to the output of the linear operation
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        # For the last layer, the sigmoid function will be the activation
        x = torch.sigmoid(self.fc3(x))
        return x"""
    
    def forward(self, x):
        # Forward pass through all layers
        for i in range(1, len(self.hidden_layer_sizes) + 2):
            #print(f'forward pass layer {i}')
            layer = getattr(self, f'fc{i}')
            x = layer(x)
            if i < len(self.hidden_layer_sizes):
                self.n_forward_calls += 1
                x = self.activation(x)
        # Apply sigmoid activation to the output layer
        self.n_forward_calls += 1
        x = torch.sigmoid(x)
        return x
    
    #
    # 1. 4. Training loop
    # For details, see Machine Learning with PyTorch and Scikit-Learn.
    #
    def train(self, num_epochs, loss_fn, optimizer, train_dl, train_size, batch_size, x_valid, y_valid):
        # Initialize weights
        self.apply(self._init_weights)
    
        n_calls_array = np.zeros(num_epochs)

        # Loss and accuracy history objects initialization
        loss_hist_train = [0] * num_epochs
        accuracy_hist_train = [0] * num_epochs
        loss_hist_valid = [0] * num_epochs
        accuracy_hist_valid = [0] * num_epochs
        delta_times = [0] * num_epochs
        self.n_forward_calls = 0
        
        # Learning loop
        for epoch in tqdm(range(num_epochs)):
            start_time = time.time()
            # Batch learn
            for x_batch, y_batch in train_dl:
                #print('*'*20)
                # ---
                # 1.4.1. Get the predictions, the [:,0] reshapes from (batch_size,1) to (batch_size)
                pred = self(x_batch)[:,0]
                # 1.4.2. Compute the loss
                loss = loss_fn(pred, y_batch)
                # 1.4.3. Back propagate the gradients
                # The `backward()` method, already available in PyTroch, calculates the 
                # derivative of the Error in respect to the NN weights
                # applying the chain rule for hidden neurons.
                loss.backward()
                # 1.4.4. Update the weights based on the computed gradients
                optimizer.step()
                # ---
                
                # Reset to zero the gradients so they will not accumulate over the mini-batches
                optimizer.zero_grad()
                
                # Update performance metrics
                loss_hist_train[epoch] += loss.item()
                is_correct = ((pred>=0.5).float() == y_batch).float()
                accuracy_hist_train[epoch] += is_correct.mean()
            
            n_calls_array[epoch] = self.n_forward_calls
            self.n_forward_calls = 0
            delta_times[epoch] = time.time() - start_time
            # Average the results
            loss_hist_train[epoch] /= train_size/batch_size
            accuracy_hist_train[epoch] /= train_size/batch_size
            
            # Predict the validation set
            pred = self(x_valid)[:, 0]
            loss_hist_valid[epoch] = loss_fn(pred, y_valid).item()
            is_correct = ((pred>=0.5).float() == y_valid).float()
            accuracy_hist_valid[epoch] += is_correct.mean()
            
        return loss_hist_train, loss_hist_valid, accuracy_hist_train, accuracy_hist_valid, n_calls_array, delta_times

    # Not needed normaly, it is just for mlextend plots
    def predict(self, x):
        print(f'predict with input shape: {x.shape}')
        x = torch.tensor(x, dtype=torch.float32)
        pred = self.forward(x)[:, 0]
        print(f'finished predict with output shape: {pred.shape}')
        return (pred>=0.5).float()
        

In [5]:
class nn_model():
    def __init__(self,
                X_train,
                X_test,
                y_train,
                y_test,
                input_size, 
                output_size,
                hidden_layer_sizes, 
                optimizer_name:str,
                num_epochs,  
                train_size,
                batch_size,
                learning_rate,
                log_path = None,
                train_dl = None,
                log_level = None,
                seed = 42,
                activation = nn.ReLU(),
                loss_fn = RMSELoss()
                ):
        

        self.X_train = X_train
        self.X_test = X_test
        self.y_train = y_train
        self.y_test = y_test
        self.input_size = input_size
        self.output_size = output_size
        self.hidden_layer_sizes = hidden_layer_sizes
        self.optimizer_name = optimizer_name
        self.num_epochs = num_epochs
        self.train_dl = train_dl 
        self.train_size = train_size
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        self.log_path = log_path
        self.seed = seed 
        self.activation = activation
        self.loss_fn = loss_fn

        
        self.arcthicture = Net_Arc(input_size=input_size,
                                  output_size=output_size,
                                  hidden_layer_sizes=hidden_layer_sizes,
                                  activation=activation)
        
        self.arcthicture.to('cpu')
        
        
        self.fitnesses, self.test_fitnesses, self.accuracy, self.accuracy_valid, self.n_forward_calls, self.delta_times = self.fit(
                X_train,
                X_test,
                y_train,
                y_test,
                num_epochs=num_epochs, 
                loss_fn=loss_fn, 
                optimizer_name=optimizer_name,
                batch_size=batch_size, 
                learning_rate=learning_rate,
                seed=seed)
    

        self.logger(self.log_path)


        self.fitness = torch.tensor(self.fitnesses[-1])
        self.test_fitness = torch.tensor(self.test_fitnesses[-1])


    def logger(self, log_path): 
        

        df = pd.DataFrame(index=range(self.num_epochs))
        df['algorithm'] = str(self.optimizer_name)
        df['Instance ID'] = 1 #PLACEHOLDER
        df['dataset'] = 2 #PLACEHOLDER 
        df['seed'] = self.seed
        df['epochs'] = range(1, self.num_epochs + 1)
        df['fitness'] = self.fitnesses
        df['running time'] = self.delta_times
        df['population nodes'] = 7 #PLACEHOLDER
        df['test_fitness'] = self.test_fitnesses
        df['Elite nodes'] = 9 #PLACEHOLDER
        df['niche entropy'] = 10 #PLACEHOLDER
        df['sd(pop.fit)'] = 11 #PLACEHOLDER
        df['Log Level'] = 12 #PLACEHOLDER
        df['params'] = 'TREZE' #PLACEHOLDER
        df['n_forward_calls'] = self.n_forward_calls


        # If 
        df.to_csv(log_path, index=False, header = False)

        
        
       

    def fit(self,X_train,X_test,y_train,y_test,
                num_epochs, loss_fn, optimizer_name, batch_size, learning_rate, seed):
        """
        Train the model with the given parameters.
        
        Parameters:
        - num_epochs: Number of epochs to train the model.
        - loss_fn: Loss function to use for training.
        - optimizer: Optimizer to use for training.
        - batch_size: Size of each batch during training.
        - x_valid: Validation input data.
        - y_valid: Validation target data.
        - learning_rate: Learning rate for the optimizer.
        
        Returns:
        - history: Training history containing loss and accuracy metrics.
        """
        
        torch.manual_seed(seed)

        # Define datasets for data loaders
        train_ds = TensorDataset(X_train, y_train)
        test_ds = TensorDataset(X_test, y_test)

        train_size = len(train_ds)
        if optimizer_name == 'GD':
            batch_size = train_size
            train_dl = DataLoader(train_ds, batch_size, shuffle=True)
            #val_dl = DataLoader(val_ds, batch_size, shuffle=True)
        
        elif optimizer_name == 'SGD':
            batch_size = 1
            train_dl = DataLoader(train_ds, batch_size, shuffle=True)
            #val_dl = DataLoader(val_ds, batch_size, shuffle=True)

        else:
            batch_size = batch_size
            train_dl = DataLoader(train_ds, batch_size, shuffle=True)
            #val_dl = DataLoader(val_ds, batch_size, shuffle=True)
        
        optimizer_choiche = {
        'GD': torch.optim.SGD(self.arcthicture.parameters(), lr=learning_rate),
        'SGD': torch.optim.SGD(self.arcthicture.parameters(), lr=learning_rate),
        'MiniSGD': torch.optim.SGD(self.arcthicture.parameters(), lr=learning_rate),
        'ASGD': torch.optim.ASGD(self.arcthicture.parameters(), lr=learning_rate),
        'RMSprop': torch.optim.RMSprop(self.arcthicture.parameters(), lr=learning_rate),
        'Adam': torch.optim.Adam(self.arcthicture.parameters(), lr=learning_rate)
        }
        optimizer_instance = optimizer_choiche[optimizer_name]
        
        return self.arcthicture.train(
            num_epochs=num_epochs, 
            loss_fn=loss_fn, 
            optimizer=optimizer_instance, 
            train_dl=train_dl, 
            train_size=train_size, 
            batch_size=batch_size,
            x_valid=X_test,     
            y_valid=y_test )
        



## Load Data

In [6]:
# Reading the desired dataset
df = pd.read_csv("../data/sustavianfeed.csv", sep=';')

# Dropping the first column (index) and renaming the columns
df = df.drop(columns= ['WING TAG', 'EMPTY MUSCULAR STOMACH'])

# Moving crude protein to the end of the dataframe
df = df[[col for col in df.columns if col != 'CRUDE PROTEIN'] + ['CRUDE PROTEIN']]

In [7]:
df

,WEIGHT,HOT CARCASS WEIGHT,CARCASS WEIGHT WITH HEAD AND LEGS,COLD CARCASS WEIGHT,BREAST WEIGHT (2),THIGH WEIGHT (2),SPLEEN,LIVER,HEART,INTESTINE,GLANDULAR STOMACH,ETHER EXTRACT,CRUDE PROTEIN
0,2223.3,1429.6,1725.6,1394.0,214.0,489.4,3.716,38.636,9.305,123.171,13.170,0.38,86.105469
1,2201.9,1450.2,1769.8,1405.4,236.0,538.7,3.494,34.725,10.084,71.800,9.781,1.66,86.143472
2,2159.9,1398.4,1724.9,1461.7,241.8,512.1,4.023,31.932,10.635,61.380,6.217,0.98,86.416898
3,2198.7,1473.9,1800.4,1425.1,227.7,549.9,3.087,32.326,11.927,64.879,8.358,1.10,85.959935
4,2003.2,1291.2,1581.6,1260.1,224.7,473.2,3.723,30.105,9.855,68.562,7.572,6.34,81.693637
...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,2633.1,1683.2,2034.1,1637.2,213.8,610.9,4.777,45.992,12.796,74.888,7.857,1.07,88.999126
92,2346.2,1547.8,1819.5,1511.2,228.0,528.2,3.673,35.090,11.504,68.455,7.837,1.13,88.507288
93,2648.2,1722.9,2050.0,1669.8,253.1,610.0,5.176,50.505,17.194,81.502,7.332,1.91,90.375587
94,2262.6,1498.5,1813.9,1468.0,212.1,548.4,2.829,30.266,11.129,53.011,5.425,2.21,91.211353


# Nested CV with Grid Search

In [10]:
seed = 42
random.seed(seed)

# Edit the name and log directory based on the model you want to run

#MODEL_NAME = 'GP'
#MODEL_NAME = 'GSGP'
#MODEL_NAME = 'SLIM-GSGP'
MODEL_NAME = 'NN'

DATASET_NAME = MODEL_NAME +'_sustavianfeed'
LOG_DIR = './log/' + MODEL_NAME + '/'

LOG_LEVEL = 2
if not os.path.exists(LOG_DIR):
    os.makedirs(LOG_DIR)

In [11]:
k_outer = 10
k_inner = 5

In [10]:
# Turning df into X and y torch.Tensors
X, y = load_pandas_df(df, X_y=True)

In [11]:
total_instances = X.shape[0]
outer_test_size = total_instances // k_outer
outer_train_size = total_instances - outer_test_size
inner_val_size = outer_train_size // k_inner
inner_train_size = outer_train_size - inner_val_size

print(f'Total Instances:\t{total_instances}\n--')
print(f'Outer Train set:\t{outer_train_size}')
print(f'Test set:\t\t{outer_test_size}\n--')
print(f'Inner Train set:\t{inner_train_size}')
print(f'Validation set:\t\t{inner_val_size}\n')

Total Instances:	96
--
Outer Train set:	87
Test set:		9
--
Inner Train set:	70
Validation set:		17



In [12]:
inner_train_size

70

In [27]:
fixed_params = {
    'input_size': X.shape[1],
    'output_size': 1,
    'num_epochs': 250,  # Example number of epochs
    'train_dl': None,  # Will be set during training, according to the optimizer
    'loss_fn': RMSELoss(),  # Example loss function
    'train_size': inner_train_size,
    'batch_size': 7,  # Example batch size
    'activation': nn.ReLU(),
   
}


param_grid = {
    'hidden_layer_sizes': [(3,4),(3, 4, 4), (5,6)],  # Example hidden layer sizes
    'optimizer_name': ['GD', 'SGD', 'MiniSGD', 'Adam', 'RMSprop', 'ASGD'],  # Example optimizers
    'learning_rate': [0.005, 0.01]  # Example learning rates,

}


In [28]:
outer_results = gp_nested_cross_validation(X, y, gp_model=nn_model, k_outer=k_outer, k_inner=k_inner, fixed_params=fixed_params, param_grid=param_grid, seed=seed, LOG_DIR=LOG_DIR, DATASET_NAME=DATASET_NAME)

Outer fold 1/10
-----
 Inner fold 1/5
Training shape: torch.Size([68, 12])
Validation shape: torch.Size([18, 12])



100%|██████████| 250/250 [00:02<00:00, 86.82it/s]


-----
 Inner fold 2/5
Training shape: torch.Size([69, 12])
Validation shape: torch.Size([17, 12])



100%|██████████| 250/250 [00:02<00:00, 83.83it/s]


-----
 Inner fold 3/5
Training shape: torch.Size([69, 12])
Validation shape: torch.Size([17, 12])



...

-----
 Inner fold 5/5
Training shape: torch.Size([70, 12])
Validation shape: torch.Size([17, 12])



100%|██████████| 250/250 [00:02<00:00, 100.96it/s]


Best inner combination: {'hidden_layer_sizes': (3, 4), 'optimizer_name': 'RMSprop', 'learning_rate': 0.005} with median RMSE: 86.39286041259766
Training best combination on entire learning set


100%|██████████| 250/250 [00:02<00:00, 87.01it/s]


          0   1   2   3    4          5         6   7          8   9   10  11  \
0    RMSprop   1   2  52    1  90.029626  0.015652   7  87.440948   9  10  11   
1    RMSprop   1   2  52    2  90.168278  0.014962   7  87.440948   9  10  11   
2    RMSprop   1   2  52    3  90.163816  0.016638   7  87.440948   9  10  11   
3    RMSprop   1   2  52    4  90.201324  0.014057   7  87.440948   9  10  11   
4    RMSprop   1   2  52    5  90.171809  0.023175   7  87.440948   9  10  11   
..       ...  ..  ..  ..  ...        ...       ...  ..        ...  ..  ..  ..   
245  RMSprop   1   2  52  246  90.131683  0.010236   7  87.440948   9  10  11   
246  RMSprop   1   2  52  247  90.153953  0.010515   7  87.440948   9  10  11   
247  RMSprop   1   2  52  248  90.149308  0.011125   7  87.440948   9  10  11   
248  RMSprop   1   2  52  249  90.203633  0.010571   7  87.440948   9  10  11   
249  RMSprop   1   2  52  250  90.199773  0.010068   7  87.440948   9  10  11   

     12                    

Saving results and configs to a .csv 

In [29]:
for i in range(k_outer):
    df = pd.read_csv(os.path.join(LOG_DIR, f'{DATASET_NAME}_outer_{i}.csv'), header=None)
    
    df.iloc[:, 13] = str(outer_results[i]['dynamic_params'])

    df.to_csv(os.path.join(LOG_DIR, f'{DATASET_NAME}_outer_{i}.csv'), index=False, header=False)

In [30]:
outer_results_df = pd.DataFrame(outer_results)
outer_results_df.to_csv(LOG_DIR+DATASET_NAME+'_outer_results.csv', index=False)


## Visualizations 

In [12]:
train_test_best_combs(model_name=MODEL_NAME)         

In [13]:
test_best_combs(model_name=MODEL_NAME)

## Brief Bloat and Overfitting Discussion 

- ...


In [17]:
fit_or_size_per_comb(k_outer,MODEL_NAME)

## Early Convergence Discussion 

- ....

## Future Work

- ....